In [5]:
def points_to_take(rep,mat):
    if rep != 'Retrieval_new_rec/':
        to_take, time = [54, 100], mat['time']-3
    else:
        to_take, time = [39, 120], mat['time']-5
    return(to_take, time)

In [9]:
"""
SUM UP CLASSIF RESULTS for POWER
new scripts with sklearn and ALL or AVERAGE values of POWER
"""
from brainpipe.system import study
import numpy as np
from os import listdir,makedirs
from os.path import isfile, join,exists
import numpy as np
from itertools import product
from brainpipe.statistics import perm_pvalue2level
from utils import rename_elecs

st = study('Ripples')
conds = ['low','high']
val = 'xpow'
reps = ['Encoding/', 'Retrieval_new_odors/', 'Retrieval_new_rec/']
average = False

path_classif = join(st.path, 'classified/{}')
clf_name = join(path_classif, '{}_LDA_clf_{}_{}_{}.npz')
path_pow = join(st.path, 'feature/{}')
pow_file = join(path_pow, '{}_cond={}_bipo_feat_norm.npz')
path2save = path_classif

subjects = ['FERJ','LEFC','SEMC','PIRJ','VACJ','CHAF']
freqs = ['HFA','ripple','low_freq']

## Create a npz file with all sources informations + POWER (subjects, coords, labels)
for rep,freq in product(reps,freqs):
    
    su_codes, s_xyz, s_channels,s_labels = np.array([]),np.array([]),np.array([]),np.array([])
    s_elec_pow0, s_elec_pow1, s_auc = np.array([]),np.array([]), np.array([])
    s_elec, s_perm = np.array([]),np.array([])
    for i,su in enumerate((subjects)): #sorted
        pow_mat = np.load(pow_file.format(rep,su,conds[0]),allow_pickle=True)
        id_freq = [i for i,f in enumerate(pow_mat['fname']) if f==freq][0]
        to_take, time = points_to_take(rep,pow_mat)
        pow0 = pow_mat['xpow'][id_freq,:,to_take[0]:to_take[1],:]
        pow1 = np.load(pow_file.format(rep,su,conds[1]))['xpow'][id_freq,:,to_take[0]:to_take[1],:]
        classif = np.load(clf_name.format(rep,su,freq,conds[0],conds[1]),allow_pickle=True)
        
        if average == True:
            pow0, pow1 = np.mean(pow0, axis=(1,2)), np.mean(pow1,axis=(1,2))
        elif average == False:
            pow0, pow1 = np.mean(pow0, axis=-1), np.mean(pow1, axis=-1)
        
        xyz, channels = classif['xyz'], classif['channels']
        auc, names, perm = classif['auc'], classif['names'], classif['perm']
        
        s_xyz = np.vstack((s_xyz, xyz)) if np.size(s_xyz) else xyz
        su_codes = np.hstack((su_codes,np.array([su]*xyz.shape[0]))) \
                                if np.size(su_codes) else np.array([su]*xyz.shape[0])
        s_elec = np.hstack((s_elec,np.arange(xyz.shape[0]))) \
                                if np.size(s_elec) else np.arange(xyz.shape[0])
        s_channels = np.hstack((s_channels, channels)) \
                                if np.size(s_channels) else channels
        s_auc = np.vstack((s_auc, auc)) if np.size(s_auc) else auc
        s_perm = np.vstack((s_perm,perm)) if np.size(s_perm) else perm
        s_labels = np.hstack((s_labels, names)) if np.size(s_labels) else names
        s_elec_pow0 = np.vstack((s_elec_pow0, pow0)) if np.size(s_elec_pow0) else pow0
        s_elec_pow1 = np.vstack((s_elec_pow1, pow1)) if np.size(s_elec_pow1) else pow1
        print(s_elec_pow0.shape, s_elec.shape, s_xyz.shape)
    print('all names', np.unique(s_labels))
    print ('shapes', 'pow1',s_elec_pow1.shape, 'pow0',s_elec_pow0.shape,'da', s_auc.shape, 
           'perm',s_perm.shape,'su_codes',su_codes.shape, 's_elec',s_elec.shape,
          's_labels',s_labels.shape,'s_channels',s_channels.shape)
    su_coord_label = {'su_codes':su_codes, 's_xyz':s_xyz, 's_labels':s_labels,
                      's_elec_pow0':s_elec_pow0,'s_elec_pow1':s_elec_pow1,
                      's_da':s_auc,'s_perm':s_perm,'s_elec':s_elec, 's_channels':s_channels,
                     's_time':time}
    file_source = 'All_subjects_sources_'+freq+'_'+conds[0]+'_'+conds[1]+'.npz'
    np.savez(path2save.format(rep)+file_source,**su_coord_label)


-> Ripples loaded
(62, 46) (62,) (62, 3)
(171, 46) (171,) (171, 3)
(214, 46) (214,) (214, 3)
(279, 46) (279,) (279, 3)
(370, 46) (370,) (370, 3)
(429, 46) (429,) (429, 3)
all names ['CD' 'Frontal' 'INS' 'MTL' 'OFC_OLF' 'PHG_FuG' 'Parietal' 'aMTL'
 'cingulate' 'occipital' 'olf' 'orbital' 'temp_cx']
shapes pow1 (429, 46) pow0 (429, 46) da (429, 46) perm (429, 46, 100) su_codes (429,) s_elec (429,) s_labels (429,) s_channels (429,)
(62, 46) (62,) (62, 3)
(171, 46) (171,) (171, 3)
(214, 46) (214,) (214, 3)
(279, 46) (279,) (279, 3)
(370, 46) (370,) (370, 3)
(429, 46) (429,) (429, 3)
all names ['CD' 'Frontal' 'INS' 'MTL' 'OFC_OLF' 'PHG_FuG' 'Parietal' 'aMTL'
 'cingulate' 'occipital' 'olf' 'orbital' 'temp_cx']
shapes pow1 (429, 46) pow0 (429, 46) da (429, 46) perm (429, 46, 100) su_codes (429,) s_elec (429,) s_labels (429,) s_channels (429,)
(62, 46) (62,) (62, 3)
(171, 46) (171,) (171, 3)
(214, 46) (214,) (214, 3)
(279, 46) (279,) (279, 3)
(370, 46) (370,) (370, 3)
(429, 46) (429,) (429, 3)

In [ ]:
"""
SUM UP CLASSIF RESULTS for RSA DIAG in TIME
new scripts with sklearn and ALL or AVERAGE values of RSA
"""
from brainpipe.system import study
import numpy as np
from os import listdir, makedirs
from os.path import isfile, join,exists
import numpy as np
from itertools import product

st = study('Olfacto')
conds = ['low','high']
val = 'rsa'
exp = 'E_R' #'Ret'
average = False

path_classif = join(st.path, 'classified/RSA_clf_diag_time_'+exp+'_by_cond/')
clf_name = join(path_classif, '{}_LDA_clf_rsa_diag_low_high.npz')
rsa_path = join(st.path,'feature/1_RSA_diag_{}_by_cond/')
rsa_name = join(rsa_path, 'rsa_diag_{}_cond_{}.npz')
path2save = join(st.path, 'figure/0_clf_rsa_diag_sklearn_time_'+exp+'_by_cond/')
subjects = ['FERJ','LEFC','SEMC','PIRJ','VACJ','CHAF']

rois_to_keep = ['ACC','aHC','pHC','IFG','Ins','MFG','OFC_olf','PHG','SFG','pPirT','Amg']

## Create a npz file with all sources informations + POWER (subjects, coords, labels)
su_codes, s_xyz, s_channels,s_labels = np.array([]),np.array([]),np.array([]),np.array([])
s_elec_pow0, s_elec_pow1, s_auc = np.array([]),np.array([]), np.array([])
s_elec, s_perm = np.array([]),np.array([])
for i,su in enumerate((subjects)): #sorted
    rsa0 = np.load(rsa_name.format(exp,su,conds[0]))['rsa'].swapaxes(1,-1)
    rsa1 = np.load(rsa_name.format(exp,su,conds[1]))['rsa'].swapaxes(1,-1)
    classif = np.load(clf_name.format(su),allow_pickle=True)

    if average == True:
        rsa0, rsa1 = np.mean(rsa0, axis=(1,2)), np.mean(rsa1,axis=(1,2))
    elif average == False:
        rsa0, rsa1 = np.mean(rsa0, axis=-1), np.mean(rsa1, axis=-1)

    xyz, channels = classif['xyz'], classif['channels']
    auc, names, perm = classif['auc'], classif['labels'], classif['perm']
    new_names = rename_elecs(names,xyz)
    idx_rois = np.isin(new_names, rois_to_keep)

    s_xyz = np.vstack((s_xyz, xyz[idx_rois])) if np.size(s_xyz) else xyz[idx_rois]
    su_codes = np.hstack((su_codes,np.array([su]*xyz[idx_rois].shape[0]))) if np.size(su_codes) else np.array([su]*xyz_all[idx_rois].shape[0])
    s_elec = np.hstack((s_elec,np.arange(xyz[idx_rois].shape[0]))) if np.size(s_elec) else np.arange(xyz[idx_rois].shape[0])
    s_channels = np.hstack((s_channels, channels[idx_rois])) if np.size(s_channels) else channels[idx_rois]
    s_auc = np.vstack((s_auc, auc[idx_rois])) if np.size(s_auc) else auc[idx_rois]
    s_perm = np.vstack((s_perm,perm[idx_rois])) if np.size(s_perm) else perm[idx_rois]
    s_labels = np.hstack((s_labels, new_names[idx_rois])) if np.size(s_labels) else new_names[idx_rois]
    s_elec_pow0 = np.vstack((s_elec_pow0, rsa0[idx_rois])) if np.size(s_elec_pow0) else rsa0[idx_rois]
    s_elec_pow1 = np.vstack((s_elec_pow1, rsa1[idx_rois])) if np.size(s_elec_pow1) else rsa1[idx_rois]

print ('shapes', 'rsa1',s_elec_pow1.shape, 'rsa0',s_elec_pow0.shape,'da', s_auc.shape, 
       'perm',s_perm.shape,'su_codes',su_codes.shape, 's_elec',s_elec.shape,
      's_labels',s_labels.shape,'s_channels',s_channels.shape)
su_coord_label = {'su_codes':su_codes, 's_xyz':s_xyz, 's_labels':s_labels,
                  'rsa0':s_elec_pow0,'rsa1':s_elec_pow1,
                  's_da':s_auc,'s_perm':s_perm,'s_elec':s_elec, 's_channels':s_channels}
file_source = 'All_subjects_sources_rsa_diag_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz'
if not exists(path2save):
    makedirs(path2save)
np.savez(path2save+file_source,**su_coord_label)


In [11]:
"""
SUM UP CLASSIF RESULTS for TPSim
new scripts with sklearn 
"""
from brainpipe.system import study
import numpy as np
from os import listdir,makedirs
from os.path import isfile, join,exists
import numpy as np
from itertools import product
from brainpipe.statistics import perm_pvalue2level

st = study('Olfacto')
#conds = ['low','high']
conds = ['early','late']
val, meth = 'tps', 'BETWEEN'
exps = ['E']#,'R','E_R'], ['Enc','Ret','Enc_Ret'] #'Ret'
fold_feats = ['Enc']

path_classif = join(st.path, 'classified/TPSim_clf_EarlyLate_theta_last_analyses/')
clf_name = join(path_classif, '{}_LDA_clf_{}_{}_{}_{}_{}_k=5.npz') #su/meth/exp/conds/freq
#path_pow = join(st.path, 'feature/TPSim_{}_By_Odor_By_Cond/TPS_by_cond/6freqs/')
path_pow = join(st.path, 'feature/TPSim_3groups_Enc/TPS_btw_thgh_time_v=1_elecs=all/')
#pow_file = join(path_pow, 'TPS_pears_{}_{}_btw_{}.npz') #su/cond/freq
pow_file = join(path_pow, 'TPS_pears_learn_{}_btw_{}_2gr.npz') #su/freq
subjects = ['FERJ','LEFC','SEMC','PIRJ','VACJ','CHAF'] #CHAF

freqs = ['theta']

## Create a npz file with all sources informations + POWER (subjects, coords, labels)
for exp, fold_feat in zip(exps,fold_feats):
    for id_freq,freq in enumerate(freqs):
        su_codes, s_xyz, s_channels,s_labels = np.array([]),np.array([]),np.array([]),np.array([])
        s_elec_pow0, s_elec_pow1, s_auc = np.array([]),np.array([]), np.array([])
        s_elec, s_perm = np.array([]),np.array([])
        for i,su in enumerate(sorted(subjects)): #sorted
            #tpsim nelecs x ncombinations
            pow0 = np.load(pow_file.format(su,freq))['tps_0']
            channels_feat = np.load(pow_file.format(su,freq))['channel']
            pow1 = np.load(pow_file.format(su,freq))['tps_1']
            classif = np.load(clf_name.format(su,meth,exp,conds[0],conds[1],freq),allow_pickle=True)

            #convert in distance for reinstatement
            pow0 = 1-pow0 if fold_feat == 'E_R' else pow0
            pow1 = 1-pow1 if fold_feat == 'E_R' else pow1

            #take tpsim averaged across trials
            pow0, pow1 = np.mean(pow0, axis=-1), np.mean(pow1, axis=-1)
            xyz, channels = classif['xyz'], classif['channels']
            idx_sel = [i for i,chan in enumerate(channels_feat) if chan in channels]
            auc, names, perm = classif['auc'], classif['names'], classif['perm']
            nelecs = xyz.shape[0]

            s_xyz = np.vstack((s_xyz, xyz)) if np.size(s_xyz) else xyz
            su_codes = np.hstack((su_codes,[su]*nelecs)) if np.size(su_codes) else [su]*nelecs
            s_elec = np.hstack((s_elec,np.arange(nelecs))) if np.size(s_elec) else np.arange(nelecs)
            s_channels = np.hstack((s_channels, channels)) if np.size(s_channels) else channels
            s_auc = np.hstack((s_auc, auc)) if np.size(s_auc) else auc
            s_perm = np.vstack((s_perm,perm)) if np.size(s_perm) else perm
            s_labels = np.hstack((s_labels, names)) if np.size(s_labels) else names
            s_elec_pow0 = np.hstack((s_elec_pow0, pow0[idx_sel])) if np.size(s_elec_pow0) else pow0[idx_sel]
            s_elec_pow1 = np.hstack((s_elec_pow1, pow1[idx_sel])) if np.size(s_elec_pow1) else pow1[idx_sel]

        print('all names', np.unique(s_labels))
        print ('shapes', 'pow1',s_elec_pow1.shape, 'pow0',s_elec_pow0.shape,'da', s_auc.shape, 
               'perm',s_perm.shape,'su_codes',su_codes.shape, 's_elec',s_elec.shape,
              's_labels',s_labels.shape,'s_channels',s_channels.shape)
        su_coord_label = {'su_codes':su_codes, 's_xyz':s_xyz, 's_labels':s_labels,
                          's_elec_pow0':s_elec_pow0,'s_elec_pow1':s_elec_pow1,
                          's_da':s_auc,'s_perm':s_perm,'s_elec':s_elec, 's_channels':s_channels}
        file_source = clf_name.format(su,meth,exp,conds[0],conds[1],freq).replace(su,'All_subjects')
        np.savez(file_source,**su_coord_label)


-> Olfacto loaded
all names ['ACC' 'Amg' 'IFG' 'Ins_olf' 'MFG' 'OFC_olf' 'PHG' 'SFG' 'aHC' 'pPirT']
shapes pow1 (230,) pow0 (230,) da (230,) perm (230, 1000) su_codes (230,) s_elec (230,) s_labels (230,) s_channels (230,)


In [48]:
path = '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/classified/TPSim_clf_all_theta/'

mat = np.load(path+'All_subjects_LDA_clf_BETWEEN_R_low_high_theta_k=5.npz')
print(mat.files)
subj = [i for i,su in enumerate(mat['su_codes']) if su == 'FERJ']
chan = [i for i,chan in enumerate(mat['s_channels'][subj]) if chan == "a2-a1"]
data = mat['s_elec_pow1'][subj][chan]
print(data)

['su_codes', 's_xyz', 's_labels', 's_elec_pow0', 's_elec_pow1', 's_da', 's_perm', 's_elec', 's_channels']
[0.57091519]
